In [1]:
import bayes3d as b
import bayes3d.genjax
import joblib
from tqdm import tqdm
import os
import jax.numpy as jnp
import jax
import numpy as np

In [2]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
intrinsics = b.Intrinsics(
    height=100,
    width=100,
    fx=500.0, fy=500.0,
    cx=50.0, cy=50.0,
    near=0.01, far=20.0
)

b.setup_renderer(intrinsics)
model_dir = os.path.join(b.utils.get_assets_dir(),"bop/ycbv/models")
meshes = []
for idx in range(1,22):
    mesh_path = os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply")
    b.RENDERER.add_mesh_from_file(mesh_path, scaling_factor=1.0/1000.0)

b.RENDERER.add_mesh_from_file(os.path.join(b.utils.get_assets_dir(), "sample_objs/cube.obj"), scaling_factor=1.0/1000000000.0)


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (128, 128, 1024)


In [4]:
importance_jit = jax.jit(b.genjax.model.importance)
key = jax.random.PRNGKey(10)

In [5]:
gt_traces = []
NUM_TRACES = 100
for scene_id in tqdm(range(NUM_TRACES)):
    filename = f"data/trace_{scene_id}.joblib"
    gt_traces.append(importance_jit(key, *joblib.load(filename))[1][1])

hb_traces = []

V_VARIANT = 0
O_VARIANT = 0
HIERARCHICAL_BAYES = True

for scene_id in tqdm(range(NUM_TRACES)):
    if HIERARCHICAL_BAYES:
        filename = f"data/inferred_hb_{scene_id}.joblib"
    else:
        filename = f"data/inferred_{V_VARIANT}_{O_VARIANT}_{scene_id}.joblib"
    hb_traces.append(importance_jit(key, *joblib.load(filename))[1][1])

100%|███████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 38.38it/s]


In [6]:
# Classification accuracy
prediction_sets = [hb_traces]
wrong_prediction = []
for pred_set in prediction_sets:
    correct = 0
    for scene_id in range(NUM_TRACES):
        gt_ids = np.array(b.genjax.get_indices(gt_traces[scene_id]))
        pred_ids = np.array(b.genjax.get_indices(pred_set[scene_id]))
        # print(pred_set[scene_id]["variance"])
        # print(pred_set[scene_id]["outlier_prob"])
        if set(pred_ids) == set(gt_ids):
            correct +=1
        else:
            wrong_prediction.append(scene_id)
            print(gt_ids, pred_ids)
            continue
    print(correct)    

[21 15 18  6] [21 15  6  7]
[21  2  4  6] [21  2  6 11]
[21 15 19 12] [21 12  1 19]
[21  1 18 16] [21  1 18  2]
[21 16 12 10] [21 10 12 17]
[21  7  3  1] [21 15  1  3]
[21  3 17  1] [21  1  3  2]
[21 11  9 10] [21 10  2  9]
[21  0 13 14] [21  0 13 20]
[21 19 10 19] [21 10 19 18]
[21 18 16 17] [21 18 17 17]
[21 18 16  0] [21  0 18 17]
[21  3  7  0] [21  0  8  8]
[21 16  0 13] [21  0 13 17]
[21  2 13 10] [21 10 13 10]
[21 16 19 11] [21 11 19 17]
[21 13 16 14] [21 14 13 17]
83


In [31]:
wrong_prediction

[0, 4, 13, 22, 35, 39, 41, 50, 52, 60, 65, 67, 71, 82, 95, 97, 99]

In [50]:
b.genjax.viz_trace_meshcat(prediction_sets[0][scene_id])

In [51]:

importance_jit = jax.jit(b.genjax.model.importance)

contact_enumerators = [b.genjax.make_enumerator([f"contact_params_{i}", "variance", "outlier_prob"]) for i in range(5)]
add_object_jit = jax.jit(b.genjax.add_object)

def c2f_contact_update(trace_, key,  number, contact_param_deltas, VARIANCE_GRID, OUTLIER_GRID):
    contact_param_grid = contact_param_deltas + trace_[f"contact_params_{number}"]
    scores = contact_enumerators[number][3](trace_, key, contact_param_grid, VARIANCE_GRID, OUTLIER_GRID)
    i,j,k = jnp.unravel_index(scores.argmax(), scores.shape)
    return contact_enumerators[number][0](
        trace_, key,
        contact_param_grid[i], VARIANCE_GRID[j], OUTLIER_GRID[k]
    )
c2f_contact_update_jit = jax.jit(c2f_contact_update, static_argnames=("number",))

In [193]:
OUTLIER_VOLUME = 100.0
VARIANCE_GRID = jnp.array([0.0001, 0.001, 0.01, 0.1])
OUTLIER_GRID = jnp.array([0.000001, 0.00001, 0.0001, 0.001])

grid_params = [
    (0.3, jnp.pi, (11,11,11)), (0.2, jnp.pi, (11,11,11)), (0.1, jnp.pi, (11,11,11)),
    (0.05, jnp.pi/3, (11,11,11)), (0.02, jnp.pi, (5,5,51)), (0.01, jnp.pi/5, (11,11,11)), (0.01, 0.0, (21,21,1)),(0.05, 0.0, (21,21,1))
]
contact_param_gridding_schedule = [
    b.utils.make_translation_grid_enumeration_3d(
        -x, -x, -ang,
        x, x, ang,
        *nums
    )
    for (x,ang,nums) in grid_params
]
key = jax.random.PRNGKey(500)


In [197]:
scene_id = 4

In [198]:
V_VARIANT = 0
O_VARIANT = 0
HIERARCHICAL_BAYES = True

if HIERARCHICAL_BAYES:
    V_GRID = VARIANCE_GRID
    O_GRID = OUTLIER_GRID
else:
    V_GRID, O_GRID = jnp.array([VARIANCE_GRID[V_VARIANT]]), jnp.array([OUTLIER_GRID[O_VARIANT]])

print(V_GRID, O_GRID)

gt_trace = importance_jit(key, *joblib.load(f"data/trace_{scene_id}.joblib"))[1][1]
print(b.genjax.get_indices(gt_trace))
b.genjax.viz_trace_meshcat(gt_trace)
choices = gt_trace.get_choices()
key, (_,trace) = importance_jit(key, choices, (jnp.arange(1), jnp.arange(22), *gt_trace.get_args()[2:-1], 100.0))
print(trace.get_score())

[1.e-04 1.e-03 1.e-02] [1.e-06 1.e-05 1.e-04 1.e-03]
[21  2  4  6]
-925.0466


In [199]:


all_all_paths = []
for _ in range(3):
    all_paths = []
    for obj_id in tqdm(range(len(b.RENDERER.meshes)-1)):
        path = []
        trace_ = add_object_jit(trace, key, obj_id, 0, 2,3)
        number = b.genjax.get_contact_params(trace_).shape[0] - 1
        path.append(trace_)
        for c2f_iter in range(len(contact_param_gridding_schedule)):
            trace_ = c2f_contact_update_jit(trace_, key, number,
                contact_param_gridding_schedule[c2f_iter], V_GRID, O_GRID)
            path.append(trace_)
        # for c2f_iter in range(len(contact_param_gridding_schedule)):
        #     trace_ = c2f_contact_update_jit(trace_, key, number,
        #         contact_param_gridding_schedule[c2f_iter], VARIANCE_GRID, OUTLIER_GRID)
        all_paths.append(
            path
        )
    all_all_paths.append(all_paths)
    
    scores = jnp.array([t[-1].get_score() for t in all_paths])
    print(scores)
    normalized_scores = b.utils.normalize_log_scores(scores)
    trace = all_paths[jnp.argmax(scores)][-1]
    b.genjax.viz_trace_meshcat(trace)

print(b.genjax.get_indices(gt_trace))
print(b.genjax.get_indices(trace))
b.genjax.viz_trace_meshcat(trace)

100%|█████████████████████████████████████████████████████████████| 21/21 [00:07<00:00,  2.92it/s]


[254036.77 277971.   267588.34 231102.5  258647.53 226914.97 231279.98
 229202.4  234577.05 222440.66 274113.66 262769.97 215490.64 231787.12
 220219.11 268383.9  220186.12 212880.19 227270.05 226705.36 227331.14]


100%|█████████████████████████████████████████████████████████████| 21/21 [00:07<00:00,  2.68it/s]


[332709.9  314199.53 333137.62 352286.16 339550.34 370638.44 373638.62
 372939.75 357440.62 365413.16 318566.5  336757.44 348237.72 363077.5
 358985.1  326686.94 363916.97 357760.4  369263.7  368405.75 370946.22]


100%|█████████████████████████████████████████████████████████████| 21/21 [00:08<00:00,  2.48it/s]


[379158.75 360443.78 379588.8  386563.16 384979.78 387000.38 384699.38
 386309.03 387045.3  386224.   364991.94 383212.16 380516.62 385535.56
 378132.   373058.   388225.84 389605.2  385168.47 384259.75 387078.38]
[21  2  4  6]
[21  1  6 17]


In [183]:
b.genjax.print_trace(trace)



    SCORE: 446075.1250000
    VARIANCE: 0.0000100
    OUTLIER_PROB 0.0010000
    


In [184]:
b.genjax.viz_trace_meshcat(gt_trace)

In [186]:
trace_alternate = all_all_paths[2][18][-1]
trace_alternate2 = all_all_paths[2][19][-1]
b.genjax.print_trace(trace_alternate)
b.genjax.print_trace(trace_alternate2)
b.genjax.viz_trace_meshcat(trace_alternate)
b.genjax.viz_trace_meshcat(trace_alternate2)


    SCORE: 446075.1250000
    VARIANCE: 0.0000100
    OUTLIER_PROB 0.0010000
    

    SCORE: 445532.4375000
    VARIANCE: 0.0000100
    OUTLIER_PROB 0.0010000
    
